## Render generated `.obj` mesh files as one interactive 3D image

In [3]:
# Import necessary libraries
import nibabel as nib
import numpy as np
from skimage import measure
import pyvista as pv
from matplotlib import cm
import matplotlib.colors as mcolors
from scipy.ndimage import gaussian_filter
import os
from glob import glob

In [7]:
# Filter subcortex
subcortex_file = 'atlas_aicha_subcortex.nii.gz'
subcortex_file_L = 'atlas_aicha_subcortex_L.nii.gz'
subcortex_file_R = 'atlas_aicha_subcortex_R.nii.gz'

if not os.path.isfile(subcortex_file):

    # Set your input and output file paths
    input_file = 'atlas_aicha.nii.gz'

    # Load the NIfTI file
    img = nib.load(input_file)
    data = img.get_fdata()

    # Create a mask for the desired range (inclusive)
    mask = (data >= 345) & (data <= 384)

    # For left hemisphere, take odd numbers between 345 and 384 inclusive
    left_mask = (data >= 345) & (data <= 384) & (data % 2 == 1)
    # For right hemisphere, take even numbers between 345 and 384 inclusive
    right_mask = (data >= 345) & (data <= 384) & (data % 2 == 0)

    # Set values outside the range to 0, keep values inside unchanged
    filtered_data = np.where(mask, data, 0)
    filtered_data_L = np.where(left_mask, data, 0)
    filtered_data_R = np.where(right_mask, data, 0)

    # Save the filtered data as a new NIfTI file
    filtered_img = nib.Nifti1Image(filtered_data, img.affine, img.header)
    filtered_img_L = nib.Nifti1Image(filtered_data_L, img.affine, img.header)
    filtered_img_R = nib.Nifti1Image(filtered_data_R, img.affine, img.header)

    # Save the filtered data as a new NIfTI file
    nib.save(filtered_img, subcortex_file)
    print(f"Filtered both hemispheres saved as {subcortex_file}")

    nib.save(filtered_img_L, subcortex_file_L)
    print(f"Filtered left hemisphere saved as {subcortex_file_L}")

    nib.save(filtered_img_R, subcortex_file_R)
    print(f"Filtered right hemisphere saved as {subcortex_file_R}")


Filtered both hemispheres saved as atlas_aicha_subcortex.nii.gz
Filtered left hemisphere saved as atlas_aicha_subcortex_L.nii.gz
Filtered right hemisphere saved as atlas_aicha_subcortex_R.nii.gz


In [17]:
%%bash 

source ~/.bashrc 
nii2mesh atlas_aicha_subcortex.nii.gz -b 1 -l 0 -q 2 -r 0.01 -s 10 -a 1 individual_meshes/atlas_aicha_subcortex.obj 

1/384
Skipping 1: no voxels with this intensity
2/384
Skipping 2: no voxels with this intensity
3/384
Skipping 3: no voxels with this intensity
4/384
Skipping 4: no voxels with this intensity
5/384
Skipping 5: no voxels with this intensity
6/384
Skipping 6: no voxels with this intensity
7/384
Skipping 7: no voxels with this intensity
8/384
Skipping 8: no voxels with this intensity
9/384
Skipping 9: no voxels with this intensity
10/384
Skipping 10: no voxels with this intensity
11/384
Skipping 11: no voxels with this intensity
12/384
Skipping 12: no voxels with this intensity
13/384
Skipping 13: no voxels with this intensity
14/384
Skipping 14: no voxels with this intensity
15/384
Skipping 15: no voxels with this intensity
16/384
Skipping 16: no voxels with this intensity
17/384
Skipping 17: no voxels with this intensity
18/384
Skipping 18: no voxels with this intensity
19/384
Skipping 19: no voxels with this intensity
20/384
Skipping 20: no voxels with this intensity
21/384
Skipping 21

We'll read in the meshes created by the `nii2mesh` program and combine them into one figure.

Note: this code will exclusively render the left hemisphere; if you want to keep both, uncomment line 8 instead of line 7.

In [27]:
# Path to the folder containing the .obj files
obj_folder = "individual_meshes"

# Find the .obj files in obj_folder
obj_files = glob(os.path.join(obj_folder, "*.obj"))

num_regions = int(len(obj_files)/2)  # Left hemisphere only
# num_regions = int(len(obj_files)) # Uncomment this line to include both hemispheres

# Define the file base for the .obj files
file_base = f"{obj_folder}/atlas_aicha_subcortex"

# List to store meshes
meshes = []

# Generate num_regions colors from the 'plasma' colormap
# You can swap 'plasma' with any other colormap available in matplotlib
# cmap = cm.get_cmap('plasma', num_regions)
cmap = cm.get_cmap('gist_rainbow', num_regions)

# I want odd numbers from 345 to 384
region_indices = np.arange(345, 385, 2)

# Shuffle the region indices for random color assignment
np.random.seed(27)
shuffled_indices = region_indices.copy()
np.random.shuffle(shuffled_indices)

# Create a mapping from original region index to random color index
index_to_color_idx = {region: i for i, region in enumerate(shuffled_indices)}

# Iterate over each .obj file and add it to the meshes list
for i in region_indices:
    file_path = f"{file_base}{i}.obj"

    # Read the .obj file into a mesh
    mesh = pv.read(file_path)

    # Assign a scalar index corresponding to a shuffled color
    color_idx = index_to_color_idx[i]
    mesh.point_data['colors'] = np.full(mesh.n_points, color_idx)
    
    meshes.append(mesh)

# Combine all meshes into a single one
combined_mesh = meshes[0]
for mesh in meshes[1:]:
    combined_mesh += mesh

# Visualize the combined mesh
plotter = pv.Plotter()
# Add the mesh with custom colors (map the region index to region_colors)
plotter.add_mesh(combined_mesh, scalars="colors", cmap=cmap, show_edges=False)

plotter.show()

/var/folders/rn/z0683sns3tz7b_k0cbf8ft8w0000gq/T/ipykernel_28589/117880928.py:19: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('gist_rainbow', num_regions)


Widget(value='<iframe src="http://localhost:59285/index.html?ui=P_0x31f9a37f0_16&reconnect=auto" class="pyvist…

In [26]:
import os
import numpy as np
import pyvista as pv
from glob import glob
from matplotlib.colors import ListedColormap

# Path to the folder containing the .obj files
obj_folder = "individual_meshes"
file_base = f"{obj_folder}/atlas_aicha_subcortex"

# I want odd numbers from 345 to 384
region_indices = np.arange(345, 385, 2)

# Define which region to highlight
highlight_index = 383

# List to store meshes
meshes = []

# Define a custom 2-color colormap: gray60 and bright red (or any highlight color)
custom_cmap = ListedColormap(["lightgray", "crimson"])  # gray for background, crimson for highlighted region

# Iterate over each .obj file and assign color index
for i in region_indices:
    file_path = f"{file_base}{i}.obj"

    # Read the .obj file into a mesh
    mesh = pv.read(file_path)

    # Set color index: 1 for highlight, 0 for others
    color_idx = 1 if i == highlight_index else 0
    mesh.point_data['colors'] = np.full(mesh.n_points, color_idx)

    meshes.append(mesh)

# Combine all meshes into a single one
combined_mesh = meshes[0]
for mesh in meshes[1:]:
    combined_mesh += mesh

# Visualize the combined mesh
plotter = pv.Plotter()
plotter.add_mesh(combined_mesh, scalars="colors", cmap=custom_cmap, show_edges=False)
plotter.show()


Widget(value='<iframe src="http://localhost:65096/index.html?ui=P_0x307e9e1c0_18&reconnect=auto" class="pyvist…